In [1]:
import math
import os
import matplotlib
import imghdr
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.xception import Xception, preprocess_input
from keras.optimizers import Adam
#from keras.preprocessing import image
import keras.utils as image
from keras.losses import categorical_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [10]:
matplotlib.use('Agg')
#dataset_root = "E://roop//"

dataset_root = 'C:\\DatasetD\\GHOST_100'
result_root = "C:\\DatasetD\\"
classes = ["real", "fake"]
num_classes = 2

epochs_pre = 10
epochs_fine = 20
batch_size_pre = 2
batch_size_fine = 2
lr_pre = 1e-3
lr_fine = 1e-4
snapshot_period_pre = 5
snapshot_period_fine = 5
split = 0.7

In [3]:
# make input_paths and labels
input_paths, labels = [], []
for class_name in os.listdir(dataset_root):
    class_root = os.path.join(dataset_root, class_name)
    class_id = classes.index(class_name)
    for path in os.listdir(class_root):
        path = os.path.join(class_root, path)
        if imghdr.what(path) is None:
            # this is not an image file
            continue
        input_paths.append(path)
        labels.append(class_id)

# convert to one-hot-vector format
labels = to_categorical(labels, num_classes=num_classes)

# convert to numpy array
input_paths = np.array(input_paths)

In [4]:
print(len(labels), len(input_paths))

200 200


In [5]:
# split dataset for training and validation purposes
border = int(len(input_paths) * split)
train_labels = labels[:border]
val_labels = labels[border:]
train_input_paths = input_paths[:border]
val_input_paths = input_paths[border:]
print("Training on %d images and labels" % (len(train_input_paths)))
print("Validation on %d images and labels" % (len(val_input_paths)))

if os.path.exists(result_root) is False:
    os.makedirs(result_root)

Training on 140 images and labels
Validation on 60 images and labels


# Model Training using Transfer Learning Technique

Since out training dataset is quite small, we apply [transfer learning](https://machinelearningmastery.com/transfer-learning-for-deep-learning/#:~:text=Transfer%20learning%20is%20a%20machine,model%20on%20a%20second%20task.) technique in creating the detector.

We use the Keras pre-trained XceptionNet model as our base model. This pre-trained model was trained on Imagenet datasets and is able to classify images into around 1000 different classes. We fine-tune this model so that it recognises real & fake human faces as well.

In [14]:
def generate_from_paths_and_labels(input_paths, labels, batch_size, input_size=(1024, 1024)):
    num_samples = len(input_paths)
    while 1:
        perm = np.random.permutation(num_samples)
        input_paths = input_paths[perm]
        labels = labels[perm]
        for i in range(0, num_samples, batch_size):
            inputs = list(map(
                lambda x: image.load_img(x, target_size=input_size),
                input_paths[i:i+batch_size]
            ))
            inputs = np.array(list(map(
                lambda x: image.img_to_array(x),
                inputs
            )))
            inputs = preprocess_input(inputs)
            yield (inputs, labels[i:i+batch_size])

In [15]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalMaxPooling2D, Dense
from tensorflow.keras.models import Model

def vgg_like(input_shape=(1024, 1024, 3), num_classes=2, num_filters=32):
    input_tensor = Input(shape=input_shape)
    x = input_tensor
    x = x / 255.0
    # Первый блок свертки
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Увеличиваем число фильтров
    num_filters *= 2
    
    # Второй блок свертки
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Увеличиваем число фильтров
    num_filters *= 2
    
    # Третий блок свертки
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Увеличиваем число фильтров
    num_filters *= 2
    
    # Четвертый блок свертки
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Финальный слой перед GlobalMaxPooling
    x = Conv2D(x.shape[-1], (5, 5), padding='same')(x)
    
    # GlobalMaxPooling
    x = GlobalMaxPooling2D()(x)
    
    # Выходной слой
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(input_tensor, output)
    return model

# Создание модели аналога VGG
model = vgg_like(input_shape=(1024, 1024, 3), num_classes=2, num_filters=32)

# Вывод информации о модели
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1024, 1024, 3)]   0         
                                                                 
 conv2d_13 (Conv2D)          (None, 1024, 1024, 32)    896       
                                                                 
 conv2d_14 (Conv2D)          (None, 1024, 1024, 32)    9248      
                                                                 
 conv2d_15 (Conv2D)          (None, 1024, 1024, 32)    9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 512, 512, 32)     0         
 2D)                                                             
                                                                 
 conv2d_16 (Conv2D)          (None, 512, 512, 64)      18496     
                                                           

In [17]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [16]:
model.compile(optimizer=Adam(lr=lr_fine),
              loss=categorical_crossentropy,
              metrics=['accuracy'])

# train
hist_fine = model.fit_generator(
    generator=generate_from_paths_and_labels(input_paths=train_input_paths,
                                            labels=train_labels,
                                            batch_size=batch_size_fine),

  steps_per_epoch=math.ceil(len(train_input_paths) / batch_size_fine),

  epochs=epochs_fine,

  validation_data=generate_from_paths_and_labels(input_paths=val_input_paths,
                                                labels=val_labels,
                                                batch_size=batch_size_fine),

  validation_steps=math.ceil(len(val_input_paths) / batch_size_fine),

  verbose=1,

  callbacks=[ModelCheckpoint(
          filepath=os.path.join(result_root,
                                'model_fine_Cust_VGG_2_{epoch}_valloss{val_loss:.3f}.h5'),
          period=snapshot_period_fine,),
  ],
)

model.save(os.path.join(result_root, 'model_Custom_Ghost_VGG_100_2.h5'))

Epoch 1/20


C:\Users\Павел\AppData\Local\Temp\ipykernel_19160\320710496.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist_fine = model.fit_generator(


70/70 [==============================] - 23s 316ms/step - loss: 0.6979 - accuracy: 0.7143 - val_loss: 0.7444 - val_accuracy: 0.0000e+00
Epoch 2/20
70/70 [==============================] - 22s 321ms/step - loss: 0.6336 - accuracy: 0.7143 - val_loss: 1.0234 - val_accuracy: 0.0000e+00
Epoch 3/20
70/70 [==============================] - 23s 334ms/step - loss: 0.6329 - accuracy: 0.7143 - val_loss: 0.8616 - val_accuracy: 0.0000e+00
Epoch 4/20
70/70 [==============================] - 23s 324ms/step - loss: 0.6281 - accuracy: 0.7143 - val_loss: 1.0940 - val_accuracy: 0.0000e+00
Epoch 5/20
70/70 [==============================] - 23s 323ms/step - loss: 0.6097 - accuracy: 0.7143 - val_loss: 1.2123 - val_accuracy: 0.0000e+00
Epoch 6/20
70/70 [==============================] - 23s 328ms/step - loss: 0.6112 - accuracy: 0.7143 - val_loss: 1.2351 - val_accuracy: 0.0000e+00
Epoch 7/20
70/70 [==============================] - 24s 338ms/step - loss: 0.6072 - accuracy: 0.7143 - val_loss: 1.0688 - val_acc

In [13]:
hist_fine = model.fit_generator(
    generator=generate_from_paths_and_labels(input_paths=train_input_paths,
                                            labels=train_labels,
                                            batch_size=batch_size_fine),

  steps_per_epoch=math.ceil(len(train_input_paths) / batch_size_fine),

  epochs=epochs_fine,

  validation_data=generate_from_paths_and_labels(input_paths=val_input_paths,
                                                labels=val_labels,
                                                batch_size=batch_size_fine),

  validation_steps=math.ceil(len(val_input_paths) / batch_size_fine),

  verbose=1,

  callbacks=[ModelCheckpoint(
          filepath=os.path.join(result_root,
                                'model_fine_Cust_VGG_{epoch}_valloss{val_loss:.3f}.h5'),
          period=snapshot_period_fine,),
  ],
)

model.save(os.path.join(result_root, 'model_Custom_Ghost_VGG_100.h5'))

Epoch 1/20


C:\Users\Павел\AppData\Local\Temp\ipykernel_19160\2157898366.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist_fine = model.fit_generator(


70/70 [==============================] - 22s 312ms/step - loss: 0.2832 - accuracy: 0.8929 - val_loss: 1.6332 - val_accuracy: 0.5000
Epoch 2/20
70/70 [==============================] - 22s 316ms/step - loss: 0.3594 - accuracy: 0.8429 - val_loss: 2.7022 - val_accuracy: 0.2667
Epoch 3/20
70/70 [==============================] - 22s 317ms/step - loss: 0.2345 - accuracy: 0.9071 - val_loss: 2.9488 - val_accuracy: 0.2333
Epoch 4/20
70/70 [==============================] - 22s 319ms/step - loss: 0.1557 - accuracy: 0.9500 - val_loss: 1.9653 - val_accuracy: 0.5500
Epoch 5/20
70/70 [==============================] - 23s 324ms/step - loss: 0.1663 - accuracy: 0.9286 - val_loss: 2.0666 - val_accuracy: 0.5167
Epoch 6/20
70/70 [==============================] - 22s 320ms/step - loss: 0.2072 - accuracy: 0.9214 - val_loss: 3.6258 - val_accuracy: 0.2333
Epoch 7/20
70/70 [==============================] - 23s 327ms/step - loss: 0.0852 - accuracy: 0.9500 - val_loss: 5.4978 - val_accuracy: 0.2667
Epoch 8/20